<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #4: Add new clips to a Zooniverse workflow</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Dec 8th, 2021</h5>

# Set up and requirements

### Import Python packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t4_utils as t4
#import utils.server_utils as server_utils

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [3]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and zoo project

In [9]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [10]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project.value)

Enter your Zooniverse user········
Enter your Zooniverse password········


# Retrieve info about zooniverse clips

In [11]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project_name = project.value, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

Retrieving subjects from Zooniverse
subjects were retrieved successfully
Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


# Retrieve info about movies hosted in the server

In [12]:
# Store info about the movies available in the server
available_movies_df = t4.retrieve_movie_info_from_server(project_name = project.value,
                                                         db_info_dict = db_info_dict)

### Select the movie you want to upload to Zooniverse

In [13]:
movie_i = t4.movie_to_upload(available_movies_df = available_movies_df)

Dropdown(description='Movie to upload:', layout=Layout(width='50%'), options=('000114 TMBL-ROV 2000 Säckenrev…

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [14]:
# Check movie hasn't been uploaded to Zooniverse
t4.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

Film_3.mp4 has not been uploaded to Zooniverse yet


### Specify the number of clips and clip length

In [15]:
clip_selection = t4.select_clip_n_len(movie_i = movie_i.value,
                                      db_info_dict = db_info_dict)

interactive(children=(Dropdown(description='Length of clips:', options=(10, 5), style=DescriptionStyle(descrip…

### Review the clips that will be created

In [16]:
t4.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

You are about to create 38 clips from Film_3.mp4
starting at 0:00:00 and ending at 0:06:23


### Create the clips

In [17]:
clips_to_upload_df = t4.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project_name = project.value)

100%|██████████| 38/38 [00:02<00:00, 13.30it/s]

clips extracted successfully


Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [18]:
t4.check_clip_size(clip_paths = clips_to_upload_df["clip_path"].unique())

Clips are a good size (below 8 MB). Ready to be uploaded to Zooniverse


File_path,Size


### Video modifications

In [19]:
# Work in progress
clip_modification = t4.select_modification()

Dropdown(description='Select clip modification:', options=(('Color_correction', {'-crf': '30', '-c:v': 'libx26…

In [20]:
# Modify the clips
clips_ready_df = t4.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.label, 
                                 modification_details = clip_modification.value)

100%|██████████| 38/38 [00:15<00:00,  2.43it/s]

Clips modified successfully


In [21]:
# Review the size of the modified clips
t4.check_clip_size(clip_paths = clips_ready_df["modif_clip_path"].unique())

Clips are a good size (below 8 MB). Ready to be uploaded to Zooniverse


File_path,Size


In [22]:
# Compare the original and modified clips
t4.compare_clips(df = clips_ready_df)

Dropdown(description='Select original clip:', layout=Layout(width='50%'), options=('0 No movie', 'Film_3.mp4_c…

Output()

### Set Zooniverse metadata

In [25]:
upload_to_zoo, sitename, created_on = t4.set_zoo_metadata(db_info_dict = db_info_dict, 
                                                          df = clips_ready_df,
                                                          project_name = project.value)

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [26]:
t4.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = str(zoo_info_dict["subjects"]["project_id"].unique()[0]))

  0%|          | 0/38 [00:00<?, ?it/s]

38_clips_Reef Säcken_1/01/2019 subject set created
uploading subjects to Zooniverse
modified_Film_3.mp4_clips/modifiedFilm_3.mp4_clip_0_10.mp4
{'upl_seconds': 0, '#clip_length': 10, '#created_on': '1/01/2019', 'VideoFilename': 'Film_3.mp4', '#siteName': 'Reef Säcken', '#clip_modification_details': "{'-crf': '30', '-c:v': 'libx264', '-c:a': 'copy', '-filter_complex': '[0:v]crop=iw:ih*(15/100):0:0,boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b0];         [0:v]crop=iw:ih*(15/100):0:ih*(95/100),boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b1];         [0:v][b0]overlay=0:0[ovr0];         [ovr0][b1]overlay=0:H*(95/100)[ovr1]', '-map': '[ovr1]', '-pix_fmt': 'yuv420p', '-preset': 'veryfast'}", 'Subject_type': 'clip', 'siteName': 'Reef Säcken', '#decimalLatitude': 59.01442972, '#decimalLongitude': 11.11670409, '#geodeticDatum': 'EPSG:4326', '#countryCode': 'SE'}


  3%|▎         | 1/38 [00:01<00:46,  1.26s/it]

subject saved
modified_Film_3.mp4_clips/modifiedFilm_3.mp4_clip_10_10.mp4
{'upl_seconds': 10, '#clip_length': 10, '#created_on': '1/01/2019', 'VideoFilename': 'Film_3.mp4', '#siteName': 'Reef Säcken', '#clip_modification_details': "{'-crf': '30', '-c:v': 'libx264', '-c:a': 'copy', '-filter_complex': '[0:v]crop=iw:ih*(15/100):0:0,boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b0];         [0:v]crop=iw:ih*(15/100):0:ih*(95/100),boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b1];         [0:v][b0]overlay=0:0[ovr0];         [ovr0][b1]overlay=0:H*(95/100)[ovr1]', '-map': '[ovr1]', '-pix_fmt': 'yuv420p', '-preset': 'veryfast'}", 'Subject_type': 'clip', 'siteName': 'Reef Säcken', '#decimalLatitude': 59.01442972, '#decimalLongitude': 11.11670409, '#geodeticDatum': 'EPSG:4326', '#countryCode': 'SE'}


  5%|▌         | 2/38 [00:02<00:42,  1.19s/it]

...

 97%|█████████▋| 37/38 [00:37<00:00,  1.01it/s]

subject saved
modified_Film_3.mp4_clips/modifiedFilm_3.mp4_clip_370_10.mp4
{'upl_seconds': 370, '#clip_length': 10, '#created_on': '1/01/2019', 'VideoFilename': 'Film_3.mp4', '#siteName': 'Reef Säcken', '#clip_modification_details': "{'-crf': '30', '-c:v': 'libx264', '-c:a': 'copy', '-filter_complex': '[0:v]crop=iw:ih*(15/100):0:0,boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b0];         [0:v]crop=iw:ih*(15/100):0:ih*(95/100),boxblur=luma_radius=min(w\\\\,h)/5:chroma_radius=min(cw\\\\,ch)/5:luma_power=1[b1];         [0:v][b0]overlay=0:0[ovr0];         [ovr0][b1]overlay=0:H*(95/100)[ovr1]', '-map': '[ovr1]', '-pix_fmt': 'yuv420p', '-preset': 'veryfast'}", 'Subject_type': 'clip', 'siteName': 'Reef Säcken', '#decimalLatitude': 59.01442972, '#decimalLongitude': 11.11670409, '#geodeticDatum': 'EPSG:4326', '#countryCode': 'SE'}


100%|██████████| 38/38 [00:38<00:00,  1.01s/it]

subject saved


Subjects uploaded to Zooniverse


In [ ]:
#END